<a href="https://colab.research.google.com/github/AmirGhnbr/Tweet-classification-tensorflow/blob/main/Tweet_classification_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This project aims to classify a particular tweet is related to disaster or not.

## make sure using GPU

In [1]:
!nvidia-smi

Sat Oct  8 21:51:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# get the helper function
!wget https://raw.githubusercontent.com/AmirGhnbr/Tweet-classification-tensorflow/main/helper.py

--2022-10-08 21:59:34--  https://raw.githubusercontent.com/AmirGhnbr/Tweet-classification-tensorflow/main/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10566 (10K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]  10.32K  --.-KB/s    in 0s      

2022-10-08 21:59:34 (60.5 MB/s) - ‘helper.py’ saved [10566/10566]



In [4]:
from helper import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Download text dataset from [Kaggle](https://www.kaggle.com/c/nlp-getting-started/data)